In [2]:
from pathlib import Path
from cpsat_autotune import tune_for_quality_within_timelimit, import_model, tune_time_to_optimal

/Users/krupke/anaconda3/envs/mo312/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:

# Automatically load all models from the 'models' directory
def load_models(directory: str) -> list:
    # model_paths = Path(directory).rglob("*.pb")
    model_paths = [Path("models/medium_hg.pb")]
    return [import_model(path) for path in model_paths]



In [4]:

models = load_models("models")

In [5]:
#tune_for_quality_within_timelimit(models[0], 3, 10, "minimize", n_samples_per_param=2, max_samples_per_param=5, n_trials=100)
tune_time_to_optimal(models[0], timelimit_in_s=10, n_samples_per_param=5, max_samples_per_param=10, n_trials=100)

[I 2024-08-14 16:30:38,709] A new study created in memory with name: no-name-b1133616-b312-4844-b360-a500076ff6d9


Computing baseline
